In [113]:
from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np
import ee
import geemap
from ipywidgets import interactive

In [128]:
Map = geemap.Map(center=(33.87, 35.9), zoom=5,add_google_map=False)
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')
Map.addLayer(countries , {}, 'Country Boundaries');
Map

Map(center=[33.87, 35.9], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(val…

In [129]:
import os
from ipyleaflet import WidgetControl

In [130]:
import os
style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

from_date = widgets.Text(
    description='From date:',
    value='2021-03-21',
    width=200,
    style=style
)

to_date = widgets.Text(
    description='To year:',
    value='2021-03-23',
    width=300,
    style=style
)

submit = widgets.Button(
    description='Submit',
    button_style='success',
    tooltip='Click me',
    style=style
)

band_1 = widgets.Dropdown(
    description='Band 1:',
    options=['','B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9',
              'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B',
              'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60'],
    value='B4',
    style=style
)

band_2 = widgets.Dropdown(
    description='Band 2:',
    options=['','B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9',
              'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B',
              'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60'],
    value='B3',
    style=style
)

band_3 = widgets.Dropdown(
    description='Band 3:',
    options=['','B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9',
              'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B',
              'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60'],
    value='B2',
    style=style
)

county_name = widgets.Text(
    description='Country name',
    value='Lebanon',
    width=350,
    style=style
)

full_widget = widgets.VBox([
    widgets.HBox([county_name]),
    widgets.HBox([from_date,to_date]),
    widgets.HBox([band_1, band_2,band_3]),
    submit
])

full_widget

In [131]:
def maskS2clouds(image):
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
    .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    return image.updateMask(mask).divide(10000)

In [132]:
# Click event handler

def submit_clicked(b):
    
    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        try:
            start_date = from_date.value
            end_date = to_date.value
            
            county=county_name.value
            
            band1=band_1.value
            band2=band_2.value
            band3=band_3.value
            
           
            countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')
            country = countries.filter(ee.Filter.eq('ADM0_NAME', county))
            
            dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
                  .filterDate(start_date, end_date) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                  .map(maskS2clouds)
            sentinel2 =dataset.mean()
            sentinel2 =sentinel2.clip(country)

            visualization = {
            'min': 0.0,
            'max': 0.3,
            'bands': [band1, band2, band3]
                }
            
            Map.addLayer(sentinel2, visualization, f'{band1}/{band2}/{band3} _ {start_date}-{end_date}')
            
            output_widget.clear_output()

            
            
            
    
        except Exception as e:
            print(e)
            print('An error occurred during computation.')
        

        Map.default_style = {'cursor': 'default'}

submit.on_click(submit_clicked)

In [133]:
print('Change new layer opacity:')
land_layer = Map.layers[-1]
land_layer.interact(opacity=(0, 1, 0.1))

Change new layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [134]:
print('Change new layer opacity:')
land_layer = Map.layers[-2]
land_layer.interact(opacity=(0, 1, 0.1))

Change new layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [135]:
style = {'description_width': 'initial'}

output_widget1 = widgets.Output(layout={'border': '1px solid black'})
output_control1 = WidgetControl(widget=output_widget1, position='bottomright')
Map.add_control(output_control1)



submit1 = widgets.Button(
    description='Download',
    button_style='success',
    tooltip='Click to download',
    style=style
)

full_widget1 = widgets.VBox([
    submit1
])

full_widget1

In [136]:
# Click event handler

def submit_clicked1(b):
    
    with output_widget1:
        output_widget1.clear_output()
        print('Downloading...')
        Map.default_style = {'cursor': 'wait'}

        try:
            start_date = from_date.value
            end_date = to_date.value
            
            county=county_name.value
            
            band1=band_1.value
            band2=band_2.value
            band3=band_3.value
            
           
            countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')
            country = countries.filter(ee.Filter.eq('ADM0_NAME', county))
            
            dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
                  .filterDate(start_date, end_date) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                  .map(maskS2clouds)
            sentinel2 =dataset.mean()
            #sentinel2 =sentinel2.clip(country)

            visualization = {
            'min': 0.0,
            'max': 0.3,
            'bands': [band1, band2, band3]
                }
            out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
            filename = os.path.join(out_dir, f'{band1}-{band2}-{band3}_{start_date}-{end_date}-Sentinel-2.tif')
            feature = Map.draw_last_feature
            if feature is None:
                geom = ee.Geometry.Polygon([[[35.429556, 33.80422],
                                             [35.429556, 33.95016],
                                             [35.674033, 33.95016],
                                             [35.674033, 33.80422],
                                             [35.429556, 33.80422]]])
                feature = ee.Feature(geom, {})
            roi = feature.geometry()
            geemap.ee_export_image(sentinel2, filename=filename, scale=90, region=roi, file_per_band=False)
            
            
            output_widget1.clear_output()

            
            
            
    
        except Exception as e:
            print(e)
            print('An error occurred during computation.')
        

        Map.default_style = {'cursor': 'default'}

submit1.on_click(submit_clicked1)